In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#https://www.kaggle.com/datasets/siddharthm1698/coursera-course-dataset/data

In [ ]:
!kaggle datasets download -d siddharthm1698/coursera-course-dataset


Dataset URL: https://www.kaggle.com/datasets/siddharthm1698/coursera-course-dataset
License(s): GPL-2.0
  0% 0.00/22.8k [00:00<?, ?B/s]
100% 22.8k/22.8k [00:00<00:00, 11.3MB/s]


In [ ]:
!unzip coursera-course-dataset.zip -d dataset/


Archive:  coursera-course-dataset.zip
  inflating: dataset/coursea_data.csv  


In [ ]:
import pandas as pd
import numpy as np


In [ ]:
df = pd.read_csv('/content/dataset/coursea_data.csv')

In [ ]:
df.head()

,Unnamed: 0,course_title,course_organization,course_Certificate_type,course_rating,course_difficulty,course_students_enrolled
0,134,(ISC)² Systems Security Certified Practitioner...,(ISC)²,SPECIALIZATION,4.7,Beginner,5.3k
1,743,A Crash Course in Causality: Inferring Causal...,University of Pennsylvania,COURSE,4.7,Intermediate,17k
2,874,A Crash Course in Data Science,Johns Hopkins University,COURSE,4.5,Mixed,130k
3,413,A Law Student's Toolkit,Yale University,COURSE,4.7,Mixed,91k
4,635,A Life of Happiness and Fulfillment,Indian School of Business,COURSE,4.8,Mixed,320k


In [ ]:
df.columns

Index(['Unnamed: 0', 'course_title', 'course_organization',
       'course_Certificate_type', 'course_rating', 'course_difficulty',
       'course_students_enrolled'],
      dtype='object')

In [ ]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [ ]:
df.columns

Index(['course_title', 'course_organization', 'course_Certificate_type',
       'course_rating', 'course_difficulty', 'course_students_enrolled'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   course_title              891 non-null    object 
 1   course_organization       891 non-null    object 
 2   course_Certificate_type   891 non-null    object 
 3   course_rating             891 non-null    float64
 4   course_difficulty         891 non-null    object 
 5   course_students_enrolled  891 non-null    object 
dtypes: float64(1), object(5)
memory usage: 41.9+ KB


In [ ]:
df.isnull().sum()

,0
course_title,0
course_organization,0
course_Certificate_type,0
course_rating,0
course_difficulty,0
course_students_enrolled,0


In [ ]:
def convert_enrollment(x):
    # Check if 'k' is in the string
    if 'k' in x:
        return float(x.replace('k', '').strip()) * 1000
    # Check if 'm' is in the string
    elif 'm' in x:
        return float(x.replace('m', '').strip()) * 1000000
    # If no suffix, convert directly to float
    else:
        return float(x)

# Apply the function to the DataFrame
df['course_students_enrolled'] = df['course_students_enrolled'].apply(convert_enrollment)

# Optional: Check the conversion
print(df['course_students_enrolled'].head())


0      5300.0
1     17000.0
2    130000.0
3     91000.0
4    320000.0
Name: course_students_enrolled, dtype: float64


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD


In [ ]:
# One-hot encode categorical features
encoder = OneHotEncoder(sparse_output=False)

# Combine features into a single DataFrame
features = pd.concat([
    df[['course_rating']],  # Numerical feature
    pd.DataFrame(encoder.fit_transform(df[['course_organization', 'course_Certificate_type', 'course_difficulty']]),
                 columns=encoder.get_feature_names_out(['course_organization', 'course_Certificate_type', 'course_difficulty']))
], axis=1)

# Calculate cosine similarity
cosine_sim = cosine_similarity(features)

# Create a DataFrame for the cosine similarity matrix
cosine_sim_df = pd.DataFrame(cosine_sim, index=df['course_title'], columns=df['course_title'])


In [ ]:
# Function to recommend courses
def recommend_courses(course_title, num_recommendations=5):
    # Check if the course title exists in the DataFrame
    if course_title not in cosine_sim_df.index:
        return f"Course '{course_title}' not found."

    # Get the pairwise similarity scores of all courses with that course
    sim_scores = cosine_sim_df[course_title]

    # Sort the courses based on the similarity scores
    sorted_courses = sim_scores.sort_values(ascending=False)

    # Get the top N most similar courses (excluding the course itself)
    recommended_courses = sorted_courses.index[1:num_recommendations + 1]

    return recommended_courses.tolist()

In [ ]:
recommendations = recommend_courses('Introduction to Genomic Technologies', num_recommendations=5)
print("Recommended Courses:", recommendations)

Recommended Courses: ['Introduction to Genomic Technologies', 'Design and Interpretation of Clinical Trials', 'A Crash Course in Data Science', 'HTML, CSS, and Javascript for Web Developers', 'Psychological First Aid']


In [ ]:
import joblib

# Assuming 'model' is your trained model (e.g., a matrix or function)
joblib.dump(predicted_ratings_df, 'coursera_edurecommender_model.pkl')


['coursera_edurecommender_model.pkl']